In [6]:
import pennylane as qml
from pennylane import numpy as np
import qiskit

## Problem description
- Implement a circuit that returns $|00\rangle$ and $|10\rangle$ with equal probability.
- Use only `CNOT`, `RX`, and `RY`.
- Find parameters with gradient descent.
- Simulations must be done with sampling and noise.

### Intuition
The circuit that returns $|00\rangle$ and $|10\rangle$ with equal probability is one that generates the state vector

\begin{equation}
|\psi\rangle = \frac{1}{\sqrt{2}} [|01\rangle + |10\rangle].
\end{equation}

Now, what is the non-parametric circuit that generates this state vector?

Well, $|\psi\rangle$ looks kind of like the simplest Bell state $|\beta_{00}\rangle = \frac{1}{\sqrt{2}} [|00\rangle + |11\rangle]$. This state, $|\beta_{00}\rangle$, says that qubit one and qubit two will be in the same compuational basis state upon measurement. We want the opposite of that: if one qubit is in the $|0\rangle$ state, the other ought to be in the $|0\rangle$ state, and vice versa. A simple way to implement the circuit that acheives our goals is to modify the familiar circuit for $|\beta_{00}\rangle$ by inserting an additional `X` gate on the first (as opposed the the zeroth) qubit.

In [97]:
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.x(1)
qc.cx(0, 1)
# qc.x(1) # alternative X-gate position

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_counts()

print(qc)
print(psi)

     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
{'01': 0.5, '10': 0.5}


Circuit matrix

In [105]:

#Changing the simulator 
backend = qiskit.Aer.get_backend('unitary_simulator')

result = qiskit.execute(qc, backend).result()

print(result.get_unitary(qc, decimals=1))

[[ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.70710678+0.00000000e+00j  0.70710678-8.65956056e-17j]
 [ 0.70710678+0.00000000e+00j -0.70710678+8.65956056e-17j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [ 0.70710678+0.00000000e+00j  0.70710678-8.65956056e-17j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.70710678+0.00000000e+00j -0.70710678+8.65956056e-17j]]


This is hadamard

In [80]:
qc = qiskit.QuantumCircuit(1)
qc.ry(np.pi/2, 0)

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_counts()

print(qc)
print(psi)

     ┌──────────┐
q_0: ┤ RY(pi/2) ├
     └──────────┘
{'0': 0.5, '1': 0.5}


In [107]:
RY = lambda theta : qiskit.circuit.library.standard_gates.RYGate(theta)
RY(np.pi/2).to_matrix()

array([[ 0.70710678+0.j, -0.70710678+0.j],
       [ 0.70710678+0.j,  0.70710678+0.j]])

This is X with phase

In [119]:
RX = lambda theta : qiskit.circuit.library.standard_gates.RXGate(theta)
RY(np.pi/4).to_matrix() @ RX(np.pi/2).to_matrix()

array([[ 0.65328148+0.27059805j, -0.27059805-0.65328148j],
       [ 0.27059805-0.65328148j,  0.65328148-0.27059805j]])

In [116]:
H = qiskit.circuit.library.standard_gates.HGate()


In [120]:
RX(np.pi/2).to_matrix()

array([[0.70710678+0.j        , 0.        -0.70710678j],
       [0.        -0.70710678j, 0.70710678+0.j        ]])

In [134]:
qc = qiskit.QuantumCircuit(2)
qc.ry(-np.pi/2, 0)
qc.ry(np.pi, 1)
qc.cx(0, 1)
# qc.x(1) # alternative X-gate position

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_statevector()

print(qc)
print(psi.round(2))

     ┌───────────┐     
q_0: ┤ RY(-pi/2) ├──■──
     └─┬────────┬┘┌─┴─┐
q_1: ──┤ RY(pi) ├─┤ X ├
       └────────┘ └───┘
[ 0.  +0.j -0.71+0.j  0.71+0.j -0.  +0.j]
